In [ ]:
# if code doesnt run make sure the click version is updated
# ! pip install --upgrade click
! pip install spacy
! python -m spacy download en_core_web_sm
! python -m spacy download en

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
from usefulclasses import Node, LinkedList, Queue, heapsort
import json
import nltk
from nltk.corpus import stopwords
import spacy
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
with open('books.json') as file:
    books = json.load(file)

In [4]:
class Book:
    def __init__(self, author: str, desc: str, genre: list, isbn: str, pages: int, rating: int, title: str, totalratings: int):
        self.author = author                    # Initialisation of the author of the book
        self.desc = desc                        # Short description of the content of the book
        self.genre = genre                      # List that provides the many genres that the book lies within 
        self.isbn = isbn                        # ID of the book, used to ease the access to the book objects
        self.pages = pages                      # Number of pages in each book, other mean of recommendation
        self.rating = rating                    # A ratiing on 5, with 5 being the most recommendable and 0 the least
        self.title = title                      # Initialisation of the title of the book
        self.totalratings = totalratings        # The total number of given ratings to the book, popularity credential
        self.rate = self.rating * self.totalratings # rate for sorting
        
    def __str__(self):
        return self.title + " by " + self.author + "\trate: " + str(self.rate)
    
    def similar_books(self, data: dict):
        pass
     

In [5]:
books_list = []
for book in books:
    author, desc, genre, isbn, pages, rating, title, totalratings = list(
        book.values())
    obj = Book(author, desc, list({g.lower() for g in genre.split(',')}), isbn, pages, rating, title, totalratings)
    books_list.append(obj)

# sort books in ascending order
heapsort(books_list)

In [13]:
books_dict = {}
stopwords_set = set(stopwords.words('english'))
for book in books_list:
    tokenized_title = list({word.lower() for word in book.title.split(' ') if word.lower() not in stopwords_set and word.isalnum()})
    tokenized_desc = list({word.lower() for word in book.desc.split(' ') if word.lower() not in stopwords_set and word.isalnum()})
    books_dict[book] = set(tokenized_title + book.genre + tokenized_desc)

In [ ]:
class user:
    def __init__(self, id: int, name: str, fav_genres: LinkedList, library: Queue, wishlist: Queue):
        self.id = id
        self.name = name
        self.fav_genres = fav_genres
        self.library = library
        self.wishlist = wishlist

    def read_book(self):
        pass

    def rate_book(self):
        pass

    def add_to_wishlist(self):
        pass

    def search_author(self, author: str) -> LinkedList:
        books = LinkedList()
        for book in books_list:
            if author in book.author:
                books.prepend(book)
        return books

    def search_book(self, title: str) -> LinkedList:
        books = LinkedList()
        for book in books_list:
            if title in book.title:
                books.prepend(book)
        return books

    def search_book_isbn(self, isbn: str) -> LinkedList:
        books = LinkedList()
        for book in books_list:
            if isbn in book.isbn:
                # prepend to get in decreasing order with low cost of operation (add at head)
                books.prepend(book)
        return books

    def search_genre(self, genre: str) -> LinkedList:
        books = LinkedList()
        for book in books_list:
            if genre.lower() in book.genre:
                # prepend to get in decreasing order with low cost of operation (add at head)
                books.prepend(book)
        return books

    def search_keywords(self, *keywords: str):
        pass

    def _find_max_count(self, data: list) -> str:
        # using count dictionary approach
        counts = dict()
        for elem in data:
            # set count to 0 if checked for the first time else add 1
            counts[elem] = counts.get(elem, 0) + 1

        max_count = 0
        for (elem, count) in counts.items():
            (max_name, max_count) = (
                elem, count) if max_count < count else (max_name, max_count)

        return max_name

    def most_checked_genre(self) -> str:
        genres = [genre for book in self.library for genre in book.genre]
        if genres != []:
            return self._find_max_count(genres)

    def most_checked_author(self) -> str:
        # same approach as genre
        authors = [
            author for book in self.library for author in book.author.split(',')]
        if authors != []:
            return self._find_max_count(authors)

    def recommend_books_genre(self) -> dict:
        books = dict()
        for genre in self.fav_genres:
            count = 3
            books[genre.val] = LinkedList()
            # traverse books_list (heap) in reverse order to get values in decreasing order
            for i in range(len(books_list) - 1, -1, -1):
                if genre.val in books_list[i].genre and count != 0:
                    books[genre.val].append(books_list[i])
                    count -= 1
        return books
